In [1]:
from google.colab import drive
drive.mount('/content/drive')

rawdata_path = '/content/drive/MyDrive/my_project/my_project1/rawdata'
data_path = '/content/drive/MyDrive/my_project/my_project1/data'
final_result = '/content/drive/MyDrive/my_project/my_project1/result/reg'
final_img = '/content/drive/MyDrive/my_project/my_project1/result/img'
# rawdata_path = 'rawdata/'
# data_path = 'data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# # 1. 나눔 폰트 다운
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
from mpl_toolkits.axes_grid1 import make_axes_locatable

import geopandas as gpd

import statsmodels.api as sm
import re
from io import StringIO

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [4]:
year_list = ['2022', '2023', '2024']

for year in year_list:
    globals()[f'temp_{year}'] = pd.read_csv(f'{data_path}/final/final_{year}.csv')

    globals()[f'temp_{year}'][f'solo_20s_{year}'] = globals()[f'temp_{year}'].apply(lambda x: (x[f'solo_20_{year}'] + x[f'solo_25_{year}']) / 2, axis = 1)
    globals()[f'temp_{year}'][f'solo_30s_{year}'] = globals()[f'temp_{year}'].apply(lambda x: (x[f'solo_30_{year}'] + x[f'solo_35_{year}']) / 2, axis = 1)

    globals()[f'temp_{year}'][f'solo_all_ratio'] = globals()[f'temp_{year}'].apply(lambda x: x[f'solo_all_{year}'] / x[f'total_all_{year}'] * 100, axis=1)
    globals()[f'temp_{year}'][f'solo_20s_ratio'] = globals()[f'temp_{year}'].apply(lambda x: x[f'solo_20s_{year}'] / x[f'solo_all_{year}'] * 100, axis=1)
    globals()[f'temp_{year}'][f'solo_30s_ratio'] = globals()[f'temp_{year}'].apply(lambda x: x[f'solo_30s_{year}'] / x[f'solo_all_{year}'] * 100, axis=1)
    globals()[f'temp_{year}'] = globals()[f'temp_{year}'].replace('-', '0')



In [5]:
depend_list = ['solo_all_ratio', 'solo_20s_ratio', 'solo_30s_ratio']
depend_name = ['all', '20', '30']

In [6]:
# reg_col = ['ADM_CD', 'solo_all_ratio', 'solo_20s_ratio',
#            '네일숍', '미용실', '애완동물',
#            '분식전문점', '한식음식점', '양식음식점', '일식음식점', '중식음식점', '치킨전문점',
#            '커피-음료', '패스트푸드점', '제과점', '호프-간이주점',
#            '유통업체수', '대형마트', '쇼핑업체수', '편의점', '슈퍼마켓',
#            '지가변동률', '지가',
#            '사업체수', '종사자수', '금융기관',
#            '대학교', '기타대학', '면적', '월평균소득',
#            '총상주인구', '20대상주인구', '총직장인구', '20대직장인구']

In [7]:
reg_col = ['ADM_CD'] + depend_list + ['네일숍', '미용실', '애완동물',
           '분식전문점', '한식음식점', '양식음식점', '일식음식점', '중식음식점', '치킨전문점',
           '커피-음료', '패스트푸드점', '제과점', '호프-간이주점',
            '대형마트', '쇼핑업체수', '편의점', '슈퍼마켓',
           '지가변동률', '지가',
           '사업체수', '종사자수',
           '대학교', '기타대학', '월평균소득' ]

drop_col = [ '면적', '월평균소득', '총상주인구', '20대상주인구', '총직장인구', '20대직장인구', '유통업체수',]

In [8]:
df_2022 = temp_2022.copy()
df_2022.head()

,ADM_CD,total_all_2022,solo_all_2022,total_20_2022,solo_20_2022,total_25_2022,solo_25_2022,total_30_2022,solo_30_2022,total_35_2022,...,총매출금액_커피-음료_2022,총매출금액_패스트푸드점_2022,총매출금액_편의점_2022,총매출금액_한식음식점_2022,총매출금액_호프-간이주점_2022,solo_20s_2022,solo_30s_2022,solo_all_ratio,solo_20s_ratio,solo_30s_ratio
0,11010530,7848.0,1854.0,582.0,183.0,687.0,220.0,779.0,244.0,758.0,...,8.715381e+09,749006078.0,4.259033e+09,2.279076e+10,1.627539e+09,201.5,224.5,23.623853,10.868393,12.108954
1,11010540,2384.0,831.0,156.0,78.0,172.0,90.0,193.0,90.0,211.0,...,4.847156e+09,7462031.0,4.966037e+08,4.833707e+09,1.334099e+08,84.0,94.5,34.857383,10.108303,11.371841
2,11010550,9116.0,1530.0,1147.0,201.0,841.0,154.0,711.0,181.0,652.0,...,4.592954e+08,107221827.0,7.261664e+08,5.864678e+08,1.911793e+07,177.5,167.5,16.783677,11.601307,10.947712
3,11010560,15369.0,1604.0,1269.0,132.0,1173.0,127.0,1172.0,144.0,1075.0,...,5.489234e+08,0.0,4.087258e+08,2.379329e+09,1.762930e+08,129.5,130.5,10.436593,8.073566,8.135910
4,11030740,18531.0,5332.0,1138.0,439.0,1829.0,748.0,2393.0,897.0,2037.0,...,2.705864e+09,228290638.0,3.617802e+09,1.480354e+10,2.165285e+09,593.5,802.0,28.773407,11.130908,15.041260


In [9]:
columns_2022 = [col.replace('_2022', '') for col in df_2022.columns]
columns_2023 = [col.replace('_2023', '') for col in columns_2022]
columns_2024 = [col.replace('_2024', '') for col in columns_2023]
df_2022.columns = columns_2024
df_2022.columns

Index(['ADM_CD', 'total_all', 'solo_all', 'total_20', 'solo_20', 'total_25',
       'solo_25', 'total_30', 'solo_30', 'total_35',
       ...
       '총매출금액_커피-음료', '총매출금액_패스트푸드점', '총매출금액_편의점', '총매출금액_한식음식점',
       '총매출금액_호프-간이주점', 'solo_20s', 'solo_30s', 'solo_all_ratio',
       'solo_20s_ratio', 'solo_30s_ratio'],
      dtype='object', length=133)

In [10]:
reg_2022 = df_2022[reg_col]
reg_2022

,ADM_CD,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,...,쇼핑업체수,편의점,슈퍼마켓,지가변동률,지가,사업체수,종사자수,대학교,기타대학,월평균소득
0,11010530,23.623853,10.868393,12.108954,13.0,30.0,10.0,127.0,413.0,183.0,...,0,11.0,0.0,4.858500,96.517917,4994,56046,0.0,0.0,5007768.0
1,11010540,34.857383,10.108303,11.371841,1.0,0.0,0.0,17.0,73.0,85.0,...,0,2.0,0.0,4.673571,96.461429,970,5118,0.0,0.0,3442534.0
2,11010550,16.783677,11.601307,10.947712,0.0,10.0,6.0,14.0,41.0,37.0,...,0,3.0,0.0,5.295000,96.764000,908,4140,1.0,0.0,3647449.0
3,11010560,10.436593,8.073566,8.135910,2.0,16.0,3.0,15.0,56.0,19.0,...,0,3.0,0.0,5.151000,96.442000,1286,4294,0.0,0.0,4504006.0
4,11030740,28.773407,11.130908,15.041260,11.0,46.0,14.0,27.0,197.0,210.0,...,0,11.0,3.0,6.127000,95.366000,3213,20440,0.0,0.0,4381088.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,11010600,28.133635,8.928571,12.142857,0.0,8.0,1.0,21.0,67.0,44.0,...,0,3.0,0.0,4.457000,96.575000,977,15449,0.0,0.0,3479766.0
418,11010610,49.126157,12.002990,8.505232,11.0,35.0,11.0,240.0,1173.0,316.0,...,7,31.0,1.0,4.757000,96.444172,17095,106826,1.0,0.0,3245251.0
419,11220630,13.926386,11.276252,12.665590,53.0,227.0,59.0,103.0,525.0,105.0,...,1,65.0,1.0,6.174000,95.269000,2720,16795,0.0,0.0,3864301.0
420,11090700,24.407020,12.764194,9.997928,52.0,309.0,61.0,112.0,654.0,30.0,...,0,51.0,0.0,4.930000,97.841000,2297,7190,0.0,1.0,2318870.0


In [11]:
df_2023 = temp_2023.copy()
df_2023.head()

,ADM_CD,total_all_2023,solo_all_2023,total_20_2023,solo_20_2023,total_25_2023,solo_25_2023,total_30_2023,solo_30_2023,total_35_2023,...,총매출금액_커피-음료_2023,총매출금액_패스트푸드점_2023,총매출금액_편의점_2023,총매출금액_한식음식점_2023,총매출금액_호프-간이주점_2023,solo_20s_2023,solo_30s_2023,solo_all_ratio,solo_20s_ratio,solo_30s_ratio
0,11010530,7845.0,1869.0,583.0,172.0,686.0,241.0,778.0,222.0,757.0,...,1.187346e+10,902523398.0,5.084405e+09,3.660644e+10,3.245160e+09,206.5,226.0,23.824092,11.048689,12.092028
1,11010540,2370.0,800.0,143.0,68.0,172.0,85.0,193.0,92.0,211.0,...,6.900190e+09,111870248.0,5.324073e+08,7.375948e+09,2.476535e+08,76.5,91.5,33.755274,9.562500,11.437500
2,11010550,9116.0,1549.0,1147.0,213.0,841.0,154.0,711.0,187.0,652.0,...,4.776125e+08,189999970.0,8.978345e+08,7.587584e+08,2.295424e+07,183.5,169.5,16.992102,11.846352,10.942544
3,11010560,15369.0,1617.0,1269.0,136.0,1173.0,123.0,1172.0,143.0,1075.0,...,9.633955e+08,284394280.0,3.962739e+08,2.456528e+09,2.872706e+08,129.5,132.0,10.521179,8.008658,8.163265
4,11030740,18463.0,5314.0,1138.0,442.0,1823.0,743.0,2384.0,881.0,2035.0,...,4.049663e+09,415945610.0,4.542987e+09,2.130260e+10,3.918752e+09,592.5,792.0,28.781888,11.149793,14.904027


In [12]:
columns_2022 = [col.replace('_2022', '') for col in df_2023.columns]
columns_2023 = [col.replace('_2023', '') for col in columns_2022]
columns_2024 = [col.replace('_2024', '') for col in columns_2023]
df_2023.columns = columns_2024
df_2023.columns

Index(['ADM_CD', 'total_all', 'solo_all', 'total_20', 'solo_20', 'total_25',
       'solo_25', 'total_30', 'solo_30', 'total_35',
       ...
       '총매출금액_커피-음료', '총매출금액_패스트푸드점', '총매출금액_편의점', '총매출금액_한식음식점',
       '총매출금액_호프-간이주점', 'solo_20s', 'solo_30s', 'solo_all_ratio',
       'solo_20s_ratio', 'solo_30s_ratio'],
      dtype='object', length=133)

In [13]:
reg_2023 = df_2023[reg_col]
reg_2023

,ADM_CD,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,...,쇼핑업체수,편의점,슈퍼마켓,지가변동률,지가,사업체수,종사자수,대학교,기타대학,월평균소득
0,11010530,23.824092,11.048689,12.092028,13.0,32.0,8.0,124.0,414.0,182.0,...,0,11.0,0.0,7.902167,98.973167,4994,56046,0.0,0.0,5007768.0
1,11010540,33.755274,9.562500,11.437500,2.0,0.0,0.0,17.0,73.0,85.0,...,0,1.0,0.0,7.840571,99.057857,970,5118,0.0,0.0,3442534.0
2,11010550,16.992102,11.846352,10.942544,0.0,10.0,7.0,14.0,38.0,37.0,...,0,4.0,0.0,8.004000,98.822000,908,4140,1.0,0.0,3647449.0
3,11010560,10.521179,8.008658,8.163265,2.0,16.0,3.0,14.0,55.0,18.0,...,0,3.0,0.0,8.009000,98.655000,1286,4294,0.0,0.0,4504006.0
4,11030740,28.781888,11.149793,14.904027,12.0,46.0,12.0,26.0,202.0,217.0,...,0,11.0,3.0,10.155000,98.676000,3213,20440,0.0,0.0,4381088.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,11010600,28.862095,9.442994,12.358573,0.0,8.0,1.0,20.0,67.0,44.0,...,0,4.0,0.0,7.389000,98.986000,977,15449,0.0,0.0,3479766.0
418,11010610,51.245969,12.514302,8.195080,10.0,34.0,13.0,238.0,1193.0,316.0,...,7,34.0,1.0,7.914103,99.068069,17095,106826,1.0,0.0,3245251.0
419,11220630,14.020878,11.360719,12.885109,55.0,234.0,63.0,93.0,512.0,118.0,...,1,69.0,1.0,10.086000,98.672000,2720,16795,0.0,0.0,3864301.0
420,11090700,23.520569,13.645079,10.161909,55.0,308.0,69.0,107.0,631.0,26.0,...,0,51.0,0.0,7.019000,99.601000,2297,7190,0.0,1.0,2318870.0


In [14]:
df_2024 = temp_2024.copy()
df_2024.head()

,ADM_CD,total_all_2024,solo_all_2024,total_20_2024,solo_20_2024,total_25_2024,solo_25_2024,total_30_2024,solo_30_2024,total_35_2024,...,총매출금액_커피-음료_2023,총매출금액_패스트푸드점_2023,총매출금액_편의점_2023,총매출금액_한식음식점_2023,총매출금액_호프-간이주점_2023,solo_20s_2024,solo_30s_2024,solo_all_ratio,solo_20s_ratio,solo_30s_ratio
0,11010530,8934.0,2200.0,569.0,182.0,705.0,252.0,791.0,262.0,749.0,...,1.187346e+10,902523398.0,5.084405e+09,3.660644e+10,3.245160e+09,217.0,242.5,24.625028,9.863636,11.022727
1,11010540,2579.0,911.0,311.0,152.0,176.0,88.0,186.0,87.0,205.0,...,6.900190e+09,111870248.0,5.324073e+08,7.375948e+09,2.476535e+08,120.0,88.0,35.323769,13.172338,9.659715
2,11010550,9226.0,1582.0,1119.0,204.0,865.0,161.0,687.0,177.0,637.0,...,4.776125e+08,189999970.0,8.978345e+08,7.587584e+08,2.295424e+07,182.5,163.5,17.147193,11.536030,10.335019
3,11010560,15667.0,1646.0,1238.0,131.0,1207.0,128.0,1133.0,137.0,1051.0,...,9.633955e+08,284394280.0,3.962739e+08,2.456528e+09,2.872706e+08,129.5,125.0,10.506159,7.867558,7.594168
4,11030740,19241.0,5509.0,1111.0,439.0,1882.0,760.0,2315.0,871.0,1991.0,...,4.049663e+09,415945610.0,4.542987e+09,2.130260e+10,3.918752e+09,599.5,782.5,28.631568,10.882193,14.204030


In [15]:
columns_2022 = [col.replace('_2022', '') for col in df_2024.columns]
columns_2023 = [col.replace('_2023', '') for col in columns_2022]
columns_2024 = [col.replace('_2024', '') for col in columns_2023]
df_2024.columns = columns_2024
df_2024.columns

Index(['ADM_CD', 'total_all', 'solo_all', 'total_20', 'solo_20', 'total_25',
       'solo_25', 'total_30', 'solo_30', 'total_35',
       ...
       '총매출금액_커피-음료', '총매출금액_패스트푸드점', '총매출금액_편의점', '총매출금액_한식음식점',
       '총매출금액_호프-간이주점', 'solo_20s', 'solo_30s', 'solo_all_ratio',
       'solo_20s_ratio', 'solo_30s_ratio'],
      dtype='object', length=133)

In [16]:
reg_2024 = df_2024[reg_col]
reg_2024

,ADM_CD,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,...,쇼핑업체수,편의점,슈퍼마켓,지가변동률,지가,사업체수,종사자수,대학교,기타대학,월평균소득
0,11010530,24.625028,9.863636,11.022727,14.0,35.0,10.0,122.0,407.0,187.0,...,0,12.0,0.0,8.764667,100.182750,4994,56046,0.0,0.0,5007768.0
1,11010540,35.323769,13.172338,9.659715,1.0,0.0,1.0,17.0,76.0,85.0,...,0,1.0,0.0,8.613000,100.133714,970,5118,0.0,0.0,3442534.0
2,11010550,17.147193,11.536030,10.335019,0.0,10.0,8.0,13.0,36.0,38.0,...,0,4.0,0.0,9.032000,100.209000,908,4140,1.0,0.0,3647449.0
3,11010560,10.506159,7.867558,7.594168,3.0,15.0,4.0,13.0,53.0,20.0,...,0,4.0,0.0,9.204000,100.254000,1286,4294,0.0,0.0,4504006.0
4,11030740,28.631568,10.882193,14.204030,14.0,48.0,13.0,24.0,199.0,236.0,...,0,11.0,3.0,11.399000,100.192000,3213,20440,0.0,0.0,4381088.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,11010600,29.180084,8.870968,12.011885,0.0,8.0,1.0,19.0,67.0,43.0,...,0,4.0,0.0,8.215000,100.184000,977,15449,0.0,0.0,3479766.0
418,11010610,48.905192,11.020078,9.381491,12.0,32.0,20.0,235.0,1219.0,320.0,...,7,29.0,1.0,8.712793,100.158517,17095,106826,1.0,0.0,3245251.0
419,11220630,13.962063,10.738148,12.281260,61.0,230.0,61.0,85.0,524.0,118.0,...,1,70.0,1.0,11.430000,100.170000,2720,16795,0.0,0.0,3864301.0
420,11090700,25.509848,12.617279,9.470289,62.0,304.0,73.0,93.0,601.0,23.0,...,0,58.0,0.0,7.376000,100.144000,2297,7190,0.0,1.0,2318870.0


# 데이터 정규화

In [17]:
### 데이터 정규화

year_list = ['2022', '2023', '2024']

for year in year_list:
    globals()[f'reg_{year}'] = globals()[f'reg_{year}'].replace('-', '0')

    std_scaler = StandardScaler()
    df_std = std_scaler.fit_transform(globals()[f'reg_{year}'])
    df_std = pd.DataFrame(df_std)
    df_std.columns = globals()[f'reg_{year}'].columns
    df_std.drop(columns = 'ADM_CD', inplace = True)

    globals()[f'reg_{year}_temp'] = df_std.copy()

reg_2022 = reg_2022_temp.copy()
reg_2023 = reg_2023_temp.copy()
reg_2024 = reg_2024_temp.copy()


In [18]:
reg_2022

,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,일식음식점,...,쇼핑업체수,편의점,슈퍼마켓,지가변동률,지가,사업체수,종사자수,대학교,기타대학,월평균소득
0,0.415566,0.085282,0.198901,-0.677852,-1.045196,-0.925207,0.940577,0.113738,1.379592,0.331708,...,-0.595712,-0.906440,-0.343338,-0.003407,-0.441216,0.814002,1.957975,-0.288565,-0.223050,1.696731
1,1.502040,-0.042339,-0.003637,-1.164200,-1.293013,-1.350150,-0.939767,-1.030905,0.246323,-0.863220,...,-0.595712,-1.165734,-0.343338,-0.222565,-0.487585,-0.669465,-0.393207,-0.288565,-0.223050,0.058449
2,-0.245995,0.208340,-0.120176,-1.204729,-1.210407,-1.095184,-0.991049,-1.138636,-0.308747,-0.937903,...,-0.595712,-1.136924,-0.343338,0.513886,-0.239215,-0.692322,-0.438358,2.543395,-0.223050,0.272927
3,-0.859866,-0.383977,-0.892781,-1.123671,-1.160844,-1.222667,-0.973955,-1.088137,-0.516898,-0.913008,...,-0.595712,-1.136924,-0.343338,0.343233,-0.503533,-0.552970,-0.431248,-0.288565,-0.223050,1.169459
4,0.913616,0.129359,1.004618,-0.758910,-0.913027,-0.755230,-0.768827,-0.613447,1.691819,0.032976,...,-0.595712,-0.906440,2.098604,1.499884,-1.386783,0.157428,0.314161,-0.288565,-0.223050,1.040804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,0.851739,-0.240419,0.208217,-1.204729,-1.226928,-1.307656,-0.871391,-1.051104,-0.227799,-0.863220,...,-0.595712,-1.136924,-0.343338,-0.479222,-0.394358,-0.666884,0.083742,-0.288565,-0.223050,0.097418
418,2.882073,0.275783,-0.791302,-0.758910,-1.003893,-0.882713,2.872204,2.672352,2.917600,1.651107,...,3.939681,-0.330229,0.470643,-0.123694,-0.501750,5.275094,4.302325,2.543395,-0.223050,-0.148041
419,-0.522344,0.153762,0.351849,0.943307,0.582135,1.157012,0.530320,0.490797,0.477603,0.680229,...,0.052202,0.649329,0.470643,1.555584,-1.466407,-0.024319,0.145883,-0.288565,-0.223050,0.499899
420,0.491312,0.403592,-0.381150,0.902778,1.259501,1.242001,0.684167,0.925088,-0.389695,0.207236,...,-0.595712,0.245982,-0.343338,0.081327,0.644857,-0.180260,-0.297549,-0.288565,4.483302,-1.117656


In [19]:
selected_var = reg_2022.columns[2:]
selected_var

Index(['solo_30s_ratio', '네일숍', '미용실', '애완동물', '분식전문점', '한식음식점', '양식음식점',
       '일식음식점', '중식음식점', '치킨전문점', '커피-음료', '패스트푸드점', '제과점', '호프-간이주점', '대형마트',
       '쇼핑업체수', '편의점', '슈퍼마켓', '지가변동률', '지가', '사업체수', '종사자수', '대학교', '기타대학',
       '월평균소득'],
      dtype='object')

In [20]:
### 분석 - OLS

selected_var = reg_2022.columns[len(depend_list):]

year_list = ['2022', '2023', '2024']

df_ols = pd.DataFrame()

for year in year_list:
    df = globals()[f'reg_{year}']
    for idx, depend in enumerate(depend_list):
        X = sm.add_constant(df[selected_var])

        # Fit the OLS model
        model = sm.OLS(df[depend], X)
        result = model.fit()

        summary = result.summary()

        temp = summary.tables[1].as_csv()

        ols_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
        ols_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']
        ols_result = ols_result[['var', 'coef', 'p']]
        ols_result['p'] = ols_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
        ols_result['coef'] = ols_result['coef'].apply(lambda x: round(x, 3))
        ols_result.columns = [f'var', f'c_{year}_{depend_name[idx]}', f'p_{year}_{depend_name[idx]}']
        ols_result['var'] = ols_result['var'].str.strip()

        globals()[f'ols_{year}_{depend}'] = ols_result

        if (year == '2022') & (idx == 0):
            df_ols = ols_result
        else:
            df_ols = df_ols.merge(ols_result,
                                        how = 'left',
                                        on = 'var')

In [21]:
ols_2022_solo_30s_ratio

,var,c_2022_30,p_2022_30
0,const,-0.000,
1,네일숍,0.308,
2,미용실,-0.054,
3,애완동물,-0.189,
4,분식전문점,0.023,
5,한식음식점,-0.362,
6,양식음식점,-0.048,
7,일식음식점,0.488,+
8,중식음식점,-0.001,
9,치킨전문점,-0.318,+


In [22]:
df_ols.to_csv(f'{final_result}/ols_all.csv', index = False)
df_ols.to_excel(f'{final_result}/ols_all.xlsx', index = False)

In [23]:
len(reg_col)

28

## 컬럼명을 영어로 바꾸는 준비
- 전진선택법, 후진제거법은 변수명 영어로

In [24]:
reg_2022_en = reg_2022.copy()

new_columns = ['var{}'.format(i) for i in range(1,len(reg_col) - len(depend_list) )]
reg_2022_en.columns = depend_list + new_columns

In [25]:
col_dict = {}

for i in range(len(df_std.columns)):
    col_dict[reg_2022_en.columns[i]] = reg_2022.columns[i]
    col_dict[reg_2022.columns[i]] = reg_2022_en.columns[i]

col_dict.update({'Intercept' : '상수',
                 '상수' : 'intercept',
                 'const' : '상수',
                 '상수' : 'const'})

# 전진선택법

In [26]:
### 전진선택법 함수

def forward_selection(data, response):
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = float('inf'), float('inf')
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {}".format(response, ' + '.join(selected + [candidate]))
            score = sm.OLS.from_formula(formula, data).fit().aic
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates[0]
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
        else:
            break
    formula = "{} ~ {}".format(response, ' + '.join(selected))
    model = sm.OLS.from_formula(formula, data).fit()
    return model


In [27]:
### 분석 - fwd

selected_var = reg_2022.columns[len(depend_list):]

df_fwd = pd.DataFrame()

for year in year_list:
    df = globals()[f'reg_{year}'].copy()
    df.columns = depend_list + new_columns

    for idx, depend in enumerate(depend_list):
        drop_list = [item for item in depend_list if item != depend]

        model = forward_selection(df.drop(columns = drop_list), depend)
        summary = model.summary()
        temp = summary.tables[1].as_csv()

        fwd_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
        fwd_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']

        fwd_result['var'] = fwd_result['var'].str.strip()
        fwd_result['coef'] = fwd_result['coef'].apply(lambda x: round(x, 3))
        fwd_result['var_mapped'] = fwd_result['var'].apply(lambda x: col_dict[x])
        fwd_result = fwd_result[['var', 'var_mapped', 'coef', 'p' ]]
        fwd_result['var_mapped'] = fwd_result['var_mapped'].str.strip()
        fwd_result['p'] = fwd_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
        fwd_result.columns = [f'var2', f'var',f'c_{year}_{depend_name[idx]}', f'p_{year}_{depend_name[idx]}']
        globals()[f'fwd_{year}_{depend}'] = fwd_result

        if (year == '2022') & (idx == 0):
            df_fwd = fwd_result
        else:
            df_fwd = df_fwd.merge(fwd_result,
                                        how = 'left',
                                        on = ['var', 'var2'])

In [28]:
df_fwd.to_csv(f'{final_result}/fwd_all.csv', index = False)
df_fwd.to_excel(f'{final_result}/fwd_all.xlsx', index = False)

In [29]:
df_fwd

,var2,var,c_2022_all,p_2022_all,c_2022_20,p_2022_20,c_2022_30,p_2022_30,c_2023_all,p_2023_all,c_2023_20,p_2023_20,c_2023_30,p_2023_30,c_2024_all,p_2024_all,c_2024_20,p_2024_20,c_2024_30,p_2024_30
0,Intercept,상수,0.000,,0.000,,-0.000,,-0.000,,0.000,,0.000,,0.000,,-0.000,,-0.000,
1,var20,사업체수,0.205,+,NaN,NaN,0.182,+,0.212,+,NaN,NaN,0.274,+,0.169,+,NaN,NaN,0.273,+
2,var24,월평균소득,-0.450,+,-0.179,+,NaN,NaN,-0.440,+,-0.179,+,NaN,NaN,-0.483,+,-0.190,+,NaN,NaN
3,var6,양식음식점,0.218,+,0.068,,NaN,NaN,0.224,+,0.081,,0.271,+,0.187,+,0.065,,0.225,+
4,var3,애완동물,-0.542,+,-0.531,+,NaN,NaN,-0.603,+,-0.629,+,NaN,NaN,-0.649,+,-0.619,+,NaN,NaN
5,var4,분식전문점,0.586,+,NaN,NaN,-0.043,,0.553,+,NaN,NaN,NaN,NaN,0.483,+,NaN,NaN,NaN,NaN
6,var22,대학교,0.129,+,0.420,+,-0.089,+,0.125,+,0.404,+,-0.085,+,0.132,+,0.403,+,-0.073,
7,var9,치킨전문점,-0.299,+,NaN,NaN,-0.270,+,-0.285,+,NaN,NaN,NaN,NaN,-0.348,+,NaN,NaN,NaN,NaN
8,var13,호프-간이주점,0.124,+,NaN,NaN,0.144,+,0.135,+,NaN,NaN,NaN,NaN,0.140,+,NaN,NaN,NaN,NaN
9,var10,커피-음료,0.069,,0.100,+,NaN,NaN,NaN,NaN,0.110,+,NaN,NaN,NaN,NaN,0.102,+,NaN,NaN


### 후진제거법

In [30]:
def backward_elimination(data, response, significance_level=0.05):
    features = data.columns.tolist()
    features.remove(response)

    while len(features) > 0:
        features_with_constant = sm.add_constant(data[features])  # Add a constant term for the intercept
        model = sm.OLS(data[response], features_with_constant).fit()
        p_values = model.pvalues.iloc[1:]  # Get p-values for the features only
        max_p_value = p_values.max()  # Find the feature with the maximum p-value
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break

    model = sm.OLS(data[response], sm.add_constant(data[features])).fit()
    return model


In [31]:
### 분석 - bwd
selected_var = reg_2022.columns[len(depend_list):]

df_bwd = pd.DataFrame()

for year in year_list:
    df = globals()[f'reg_{year}'].copy()

    df.columns = depend_list + new_columns


    for idx, depend in enumerate(depend_list):
        drop_list = [item for item in depend_list if item != depend]

        model = backward_elimination(df.drop(columns = drop_list), depend)
        summary = model.summary()
        temp = summary.tables[1].as_csv()

        bwd_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
        bwd_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']

        bwd_result['var'] = bwd_result['var'].str.strip()
        bwd_result['coef'] = bwd_result['coef'].apply(lambda x: round(x, 3))
        bwd_result['var_mapped'] = bwd_result['var'].apply(lambda x: col_dict[x])
        bwd_result = bwd_result[['var', 'var_mapped', 'coef', 'p' ]]
        bwd_result['var_mapped'] = bwd_result['var_mapped'].str.strip()
        bwd_result['p'] = bwd_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
        bwd_result.columns = [f'var2', f'var',f'c_{year}_{depend_name[idx]}', f'p_{year}_{depend_name[idx]}']

        globals()[f'bwd_{year}_{depend}'] = bwd_result

        if (year == '2022') & (idx == 0):
            df_bwd = bwd_result
        else:
            df_bwd = df_bwd.merge(bwd_result,
                                        how = 'left',
                                       on = ['var', 'var2'])

In [32]:
df_bwd.to_csv(f'{final_result}/bwd_all.csv', index = False)
df_bwd.to_excel(f'{final_result}/bwd_all.xlsx', index = False)

In [33]:
df_bwd

,var2,var,c_2022_all,p_2022_all,c_2022_20,p_2022_20,c_2022_30,p_2022_30,c_2023_all,p_2023_all,c_2023_20,p_2023_20,c_2023_30,p_2023_30,c_2024_all,p_2024_all,c_2024_20,p_2024_20,c_2024_30,p_2024_30
0,const,상수,0.000,,0.000,,-0.000,,-0.000,,0.000,,0.000,,0.000,,-0.000,,-0.000,
1,var3,애완동물,-0.515,+,-0.549,+,NaN,NaN,-0.576,+,-0.674,+,-0.235,+,-0.655,+,-0.658,+,NaN,NaN
2,var4,분식전문점,0.614,+,NaN,NaN,NaN,NaN,0.624,+,NaN,NaN,NaN,NaN,0.516,+,NaN,NaN,NaN,NaN
3,var6,양식음식점,0.223,+,NaN,NaN,NaN,NaN,0.207,+,NaN,NaN,NaN,NaN,0.185,+,NaN,NaN,NaN,NaN
4,var9,치킨전문점,-0.326,+,NaN,NaN,-0.220,+,-0.270,+,NaN,NaN,-0.246,+,-0.340,+,NaN,NaN,-0.228,+
5,var13,호프-간이주점,0.117,+,NaN,NaN,0.141,+,0.114,+,NaN,NaN,NaN,NaN,0.138,+,NaN,NaN,0.137,+
6,var20,사업체수,0.213,+,NaN,NaN,0.187,+,0.236,+,NaN,NaN,0.196,+,0.209,+,NaN,NaN,0.250,+
7,var22,대학교,0.131,+,0.419,+,-0.098,+,0.128,+,0.409,+,-0.107,+,0.129,+,0.421,+,-0.080,+
8,var24,월평균소득,-0.453,+,-0.159,+,NaN,NaN,-0.452,+,-0.145,+,NaN,NaN,-0.479,+,-0.158,+,NaN,NaN


# 각연도별 통합

In [34]:
selected_var = reg_2022.columns[2:]

year_list = ['2022', '2023', '2024']

df_bwd = pd.DataFrame()

for year in year_list:

    for idx, depend in enumerate(depend_list):

        ols = globals()[f'ols_{year}_{depend}']
        fwd = globals()[f'fwd_{year}_{depend}']
        bwd = globals()[f'bwd_{year}_{depend}']

        ols['var'] = ols['var'].apply(lambda x: x if x != 'const' else '상수')

        temp = ols.merge(fwd.drop(columns = 'var2'),
                    how = 'outer',
                    on = 'var',
                    suffixes=('_ols', '_fwd'))
        temp = temp.merge(bwd.drop(columns = 'var2'),
                    how = 'outer',
                    on = 'var',
                    suffixes=('_fwd', '_bwd'))

        temp = temp.sort_values(by=[f'c_{year}_{depend_name[idx]}_fwd', f'p_{year}_{depend_name[idx]}'])
        temp = temp[temp['var'] != '상수']

        globals()[f'result_{year}_{depend}']= temp

        globals()[f'result_{year}_{depend}'].to_csv(f'{final_result}/{year}_{depend}.csv', index = False)
        globals()[f'result_{year}_{depend}'].to_excel(f'{final_result}/{year}_{depend}.xlsx', index = False)

        globals()[f'result_{year}_{idx}'] = temp

In [35]:
result_2022_2

,var,c_2022_30_ols,p_2022_30_ols,c_2022_30_fwd,p_2022_30_fwd,c_2022_30,p_2022_30
5,한식음식점,-0.362,,-0.347,+,-0.335,+
12,제과점,-0.259,+,-0.311,+,-0.180,+
19,지가,-0.249,+,-0.277,+,-0.260,+
9,치킨전문점,-0.318,+,-0.270,+,-0.220,+
22,대학교,-0.090,+,-0.089,+,-0.098,+
17,슈퍼마켓,-0.066,,-0.071,,NaN,NaN
4,분식전문점,0.023,,-0.043,,NaN,NaN
14,대형마트,0.074,,0.073,,NaN,NaN
13,호프-간이주점,0.171,+,0.144,+,0.141,+
20,사업체수,0.269,+,0.182,+,0.187,+
